In [14]:
%load_ext autoreload
%autoreload 2

from src.models.pipeline.synthetic_trainer import SyntheticTrainer
import src.data.synthetic.gengraph as gengraph
import src.models.gps as gps
from torch import set_printoptions

set_printoptions(threshold=10000)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
synthetic_trainer = SyntheticTrainer(gengraph.gen_syn1, nb_shapes=50, width_basis=100)
synthetic_trainer.assign_encodings()
model = synthetic_trainer.initialize_model(pe_channels=5, hidden_channels=4, num_layers=3, num_attention_heads=2, observe_attention=True)
graph = synthetic_trainer.get_graph()
graph

Data(edge_index=[2, 1666], feat=[350], num_nodes=350, y=[350], train_mask=[350], val_mask=[350], test_mask=[350], x=[350, 1], laplacian_eigenvector_pe=[350, 5])

In [18]:
w = gps.train(model, graph, epochs=100)
train_acc, test_acc = gps.test(model, graph)
print("Train Accuracy:", train_acc)
print("Test Accuracy:", test_acc)

100%|██████████| 100/100 [01:30<00:00,  1.11it/s]


Train Accuracy: 0.6571428775787354
Test Accuracy: 0.3795918367346939


In [21]:
w

[tensor([[[0.0028, 0.0028, 0.0029,  ..., 0.0025, 0.0025, 0.0025],
          [0.0028, 0.0028, 0.0029,  ..., 0.0025, 0.0025, 0.0025],
          [0.0028, 0.0028, 0.0029,  ..., 0.0025, 0.0025, 0.0025],
          ...,
          [0.0028, 0.0028, 0.0029,  ..., 0.0025, 0.0025, 0.0025],
          [0.0028, 0.0028, 0.0029,  ..., 0.0025, 0.0025, 0.0025],
          [0.0028, 0.0028, 0.0029,  ..., 0.0025, 0.0025, 0.0025]]],
        grad_fn=<MeanBackward1>),
 tensor([[[2.8052e-02, 1.1972e-05, 4.2225e-05,  ..., 1.1401e-07,
           3.0270e-08, 1.0222e-08],
          [4.6831e-02, 3.1047e-03, 4.5526e-03,  ..., 8.5496e-04,
           5.5302e-04, 4.0234e-04],
          [6.5340e-02, 1.9486e-03, 2.9957e-03,  ..., 4.7499e-04,
           2.7321e-04, 1.7907e-04],
          ...,
          [1.9027e-03, 2.2363e-03, 2.3218e-03,  ..., 2.8593e-03,
           3.0288e-03, 3.2216e-03],
          [1.0485e-02, 3.4507e-03, 4.4195e-03,  ..., 1.8459e-03,
           1.6918e-03, 1.5728e-03],
          [2.9878e-02, 2.4327e-03